In [ ]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

business_obj = {
    "business_id": "123",
    "name": "Bob's Liquor",
    "city": "San Mateo",
    "state": "CA",
    "categories": ["Liquor", "Lottery", "UPS Locker"]
}

with driver.session() as session:
    session.run('''
        WITH {business} AS business
        CREATE (b:Business {business_id: business.business_id})
        SET b.name = business.name,
            b.city = business.city
        WITH *
        UNWIND business.categories AS cat
        MERGE (c:Category {name: cat})
        MERGE (b)-[:IN_CATEGORY]->(c)
    ''', parameters={'business': business_obj}).consume()

In [ ]:
from neo4j.v1 import GraphDatabase
import json

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

BUSINESS_FILE = "../data/yelp_academic_dataset_business.json"

with open(BUSINESS_FILE, 'r') as file:
    with driver.session() as session:
        for line in file:
            item = json.loads(line)
        
            session.run('''
                WITH {business} AS business
                CREATE (b:Business {business_id: business.business_id})
                SET b.name = business.name,
                    b.city = business.city
                WITH *
                UNWIND business.categories AS cat
                MERGE (c:Category {name: cat})
                MERGE (b)-[:IN_CATEGORY]->(c)
            ''', parameters={'business': item}).consume()

In [ ]:
from neo4j.v1 import GraphDatabase
import json

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

BUSINESS_FILE = "../data/yelp_academic_dataset_business.json"

with open(BUSINESS_FILE, 'r') as file:
    with driver.session() as session:
        count = 0
        tx = session.begin_transaction()
        for line in file:
            item = json.loads(line)
            tx.run('''
                WITH {business} AS business
                CREATE (b:Business {business_id: business.business_id})
                SET b.name = business.name,
                    b.city = business.city
                WITH *
                UNWIND business.categories AS cat
                MERGE (c:Category {name: cat})
                MERGE (b)-[:IN_CATEGORY]->(c)
            ''', parameters={'business': item})
            count += 1
            if count > 1000:
                tx.commit()
                tx = session.begin_transaction()
                count = 0
        tx.commit()
                

In [ ]:
from neo4j.v1 import GraphDatabase
import json

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

BUSINESS_FILE = "../data/yelp_academic_dataset_business.json"

with open(BUSINESS_FILE, 'r') as file:
    with driver.session() as session:
        count = 0
        items = []
        for line in file:
            item = json.loads(line)
            items.append(item)
            count += 1
            if count > 1000:
                session.run('''
                    WITH {businesses} AS businesses
                    UNWIND businesses AS business
                    CREATE (b:Business {business_id: business.business_id})
                    SET b.name = business.name,
                        b.city = business.city
                    WITH *
                    UNWIND business.categories AS cat
                    MERGE (c:Category {name: cat})
                    MERGE (b)-[:IN_CATEGORY]->(c)
                ''', parameters={'businesses': items}).consume()
                count = 0
        
        # run query again for partial remainder...
        
                

In [ ]:
from neo4j.v1 import GraphDatabase
import json, queue, threading

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

BUSINESS_FILE = "../data/yelp_academic_dataset_business.json"

INSERT_QUERY = '''
WITH {businesses} AS businesses
                    UNWIND businesses AS business
                    CREATE (b:Business {business_id: business.business_id})
                    SET b.name = business.name,
                        b.city = business.city
                    WITH *
                    UNWIND business.categories AS cat
                    MERGE (c:Category {name: cat})
                    MERGE (b)-[:IN_CATEGORY]->(c)
'''

# init the work queue
global q 
q = queue.LifoQueue()

def run_query():
    while True:
        items = q.get()
        with driver.session() as session:
            session.run(INSERT_QUERY, parameters={'businesses': items}).consume()
            q.task_done()

for i in range(16):
    t = threading.Thread(target=run_query)
    t.daemon = True
    t.start()
            
with open(BUSINESS_FILE, 'r') as file:
    with driver.session() as session:
        count = 0
        items = []
        for line in file:
            items.append(json.loads(line))
            count += 1
            if count > 1000:
                q.put(items)
                items = []
                count = 0
            q.put(items)
    
q.join()

In [ ]:
from neo4j.v1 import GraphDatabase, CypherError
import json, queue, threading

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

BUSINESS_FILE = "../data/yelp_academic_dataset_business.json"

INSERT_QUERY = '''
WITH {businesses} AS businesses
                    UNWIND businesses AS business
                    CREATE (b:Business {business_id: business.business_id})
                    SET b.name = business.name,
                        b.city = business.city
                    WITH *
                    UNWIND business.categories AS cat
                    MERGE (c:Category {name: cat})
                    MERGE (b)-[:IN_CATEGORY]->(c)
'''

# init the work queue
global q 
q = queue.LifoQueue()

global single_threaded_q
single_threaded_q = queue.LifoQueue()



def run_query():
    while True:
        parts = q.get()
        with driver.session() as session:
            try:
                session.run(INSERT_QUERY, parameters={'businesses': parts[0]}).consume()
                q.task_done()
            except CypherError as e:
                if parts[1] < 2:
                    q.put((parts[0], parts[1]+1))
                else:
                    single_threaded_q.put((parts[0], parts[1]+1))
                q.task_done()

def run_single_threaded():
    while True:
        parts = single_threaded_q.get()
        with driver.session() as session:
            session.run(INSERT_QUERY, parameters={'businesses': parts[0]}).consume()
            single_threaded_q.task_done()

for i in range(16):
    t = threading.Thread(target=run_query)
    t.daemon = True
    t.start()

with open(BUSINESS_FILE, 'r') as file:
    with driver.session() as session:
        count = 0
        items = []
        for line in file:
            items.append(json.loads(line))
            count += 1
            if count > 1000:
                #     (params, number of retries)
                q.put((items, 0))
                items = []
                count = 0
            q.put((items, 0))
    
q.join()

t = threading.Thread(target=run_single_threaded)
t.daemon = True
single_threaded_q.join()